<a href="https://colab.research.google.com/github/cxianyao/mistral7b_rag/blob/main/mistral7b_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pypdf
# !pip install -q python-dotenv
!pip install -q transformers
!pip install -q llama-index
!pip install -q sentence-transformers
!pip install -q langchain
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.5/966.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 r

Metal(MPS): !CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install llama-cpp-python  
Nvidia GPU: !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [2]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 73.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.23-cp310-cp310-manylinux_2_35_x86_64.whl size=8125300 sha256=2e5a5cdf60f5f8395aabc8b6529763f4bcf162d6c67d2dfb987136cdcb3da782
  Stored in directory: /tmp/pip-ephem-wheel-cache-oaxtn3wo/wheels/8b/7c/23/5851b51f3b9088d6f7a5ba6d27b3494aa4940bf291b43ee04d
Successfully built llama-cpp-python


In [3]:
from google.colab import drive
import os

In [4]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
folder_id = '/Data/'  # Replace with the actual folder ID from the URL
folder_path = f'/content/drive/My Drive/{folder_id}'

# List contents of the folder
folder_contents = os.listdir(folder_path)
print(folder_contents)

['how_to_become_a_freelance_data_scientist_by_PauLabartaBajo_revised.pdf', 'Data_Cleaning_and_Preprocessing_for_Data_Science_Beginners_Data_Science_Horizons.pdf']


In [6]:
import logging
import sys
import gradio as gr

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.embeddings import LangchainEmbedding
from llama_index import ServiceContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [7]:
documents = SimpleDirectoryReader("/content/drive/My Drive/Data/").load_data()

In [8]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

total size (MB): 4368.44


4167it [00:29, 140.98it/s]                          
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


Embedding model:  
BAAI/bge-small-en-v1.5  
thenlper/gte-large

In [9]:
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)

service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine = index.as_query_engine()
response = query_engine.query("What are the sources?")

In [11]:
print(response)

 Based on the given context information, it appears that there are two sources mentioned:

1. "Data_Cleaning_and_Preprocessing_for_Data_Science_Beginners_Data_Science_Horizons.pdf" file located at "/content/drive/My Drive/Data/"
2. The book "Easy Data Analysis" by Roger D. Peng and Elizabeth Matsui


In [12]:
'''
while True:
  query=input()
  response = query_engine.query(query)
  print(response)
'''

'\nwhile True:\n  query=input()\n  response = query_engine.query(query)\n  print(response)\n'

Gradio interface:

In [13]:
def predict(input, history):
  response = query_engine.query(input)
  return str(response)

In [14]:
import gradio as gr

In [15]:
gr.ChatInterface(predict).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://478a66c217bbe17568.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [16]:
'''
import pickle

# Path to save the query_engine pickle file in Google Drive
file_path = '/content/drive/My Drive/query_engine.pkl'

# Serialize and save the query_engine object to a file using pickle
with open(file_path, 'wb') as file:
    pickle.dump(query_engine, file)

print("Query engine saved to Google Drive.")
'''

'\nimport pickle\n\n# Path to save the query_engine pickle file in Google Drive\nfile_path = \'/content/drive/My Drive/query_engine.pkl\'\n\n# Serialize and save the query_engine object to a file using pickle\nwith open(file_path, \'wb\') as file:\n    pickle.dump(query_engine, file)\n\nprint("Query engine saved to Google Drive.")\n'